## Data Cleaning and Wrangling

In [1]:
import pandas as pd

df_quotes = pd.read_csv('data/quotes.csv', na_values='nan')
df_ppl = pd.read_csv('data/people.csv', index_col='person_id')


In [2]:
df_quotes.head(10)

,person_id,car_id,budd_quote,hudd_quote,wool_quote,real_quote
0,1,1,3845.0,4120.0,3356.0,3555.0
1,1,2,6191.0,5408.0,4989.0,5285.0
2,1,3,NaN,8134.0,6841.0,7246.0
3,1,4,NaN,6005.0,5327.0,5643.0
4,1,5,3778.0,3822.0,4359.0,4617.0
5,1,6,5859.0,3536.0,3402.0,3603.0
6,1,7,3770.0,2847.0,2661.0,2818.0
7,1,8,4527.0,3703.0,2985.0,3162.0
8,1,9,4106.0,2897.0,2704.0,2864.0
9,1,10,NaN,NaN,NaN,NaN


In [3]:
df_ppl.sample(5)

,name,age,gender
person_id,,,
105,Christine,63,female
68,Randy,41,male
99,Shannon,60,female
1,Pam,18,female
141,Christal,86,female


Remove the car with id 10, too many missing values

In [4]:
car10_fltr = (df_quotes['car_id'] == 10)
df_quotes.drop(index=df_quotes[car10_fltr].index, inplace=True)
df_quotes.head(10)

,person_id,car_id,budd_quote,hudd_quote,wool_quote,real_quote
0,1,1,3845.0,4120.0,3356.0,3555.0
1,1,2,6191.0,5408.0,4989.0,5285.0
2,1,3,NaN,8134.0,6841.0,7246.0
3,1,4,NaN,6005.0,5327.0,5643.0
4,1,5,3778.0,3822.0,4359.0,4617.0
5,1,6,5859.0,3536.0,3402.0,3603.0
6,1,7,3770.0,2847.0,2661.0,2818.0
7,1,8,4527.0,3703.0,2985.0,3162.0
8,1,9,4106.0,2897.0,2704.0,2864.0
10,2,1,3740.0,5446.0,4171.0,4415.0


Join people and quotes dataframes

In [5]:
df = pd.merge(left=df_ppl, right=df_quotes, on='person_id')
df.sample(6)

,person_id,name,age,gender,car_id,budd_quote,hudd_quote,wool_quote,real_quote
759,85,Allison,52,female,4,1291.0,1768.0,1967.0,2088.0
1390,155,Erika,94,female,5,1909.0,2207.0,3044.0,3224.0
714,80,Marvin,48,male,4,1322.0,1835.0,2088.0,2216.0
1066,119,Diana,72,female,5,956.0,1146.0,1627.0,1727.0
545,61,Michelle,38,female,6,1047.0,1153.0,1484.0,1574.0
347,39,Joan,24,female,6,1317.0,1818.0,2137.0,2266.0


Get the average quote per driver and car combination

In [6]:
quotes_cols = df.loc[:, 'budd_quote':'real_quote']
df['mean_quote_percar'] = quotes_cols.mean(axis=1)
df.sample(6)

,person_id,name,age,gender,car_id,budd_quote,hudd_quote,wool_quote,real_quote,mean_quote_percar
1264,141,Christal,86,female,5,1536.0,1851.0,2907.0,3080.0,2343.50
1029,115,Zulma,69,female,4,1357.0,1618.0,1911.0,2006.0,1723.00
1205,134,Rodney,81,male,9,1337.0,1147.0,1434.0,1507.0,1356.25
1401,156,David,94,male,7,1655.0,1766.0,1751.0,1855.0,1756.75
1271,142,Allen,86,male,3,2191.0,3141.0,4341.0,4599.0,3568.00
341,38,Wilson,22,male,9,1219.0,NaN,2043.0,2165.0,1809.00


Get average quote per driver

In [7]:
# Create a series with the mean of all quotes for each driver
mean_quote_bydriver = df.groupby('person_id')['mean_quote_percar'].mean()
# Remove columns we no longer need
cols_to_drop = ['name', 'car_id', 'budd_quote', 'hudd_quote', 'wool_quote', 'real_quote', 'mean_quote_percar']
df.drop(cols_to_drop, axis=1, inplace=True)
# Remove duplicate rows from dataframe, leaving just 1 row per person
df.drop_duplicates(subset='person_id', keep='first', inplace=True)
# Set the index to the person id
df.set_index('person_id', inplace=True)
# Add the mean quotes to the dataframe
df['mean_quote'] = mean_quote_bydriver
df

,age,gender,mean_quote
person_id,,,
1,18,female,4473.064815
2,18,male,5417.351852
3,19,female,3516.472222
4,19,male,4139.018519
5,20,female,3104.675926
...,...,...,...
160,96,male,3397.111111
161,97,female,3532.000000
162,97,male,3419.861111


In [9]:
df.to_csv('data/quotesByAgeGender.csv')